In [64]:
import numpy as np
import math as m
import random as r
import cvxpy as cp
from scipy.integrate import odeint
import seaborn as sns
import KuramotoFunctionBank as KFB
import importlib
importlib.reload(KFB)

<module 'KuramotoFunctionBank' from '/Users/Moncho/Desktop/Year 1/kuramotocontrol/KuramotoFunctionBank.py'>

In [39]:
#Parameter control center
N=7

#These are phases differences w.r.t osc 1
x_min = np.array([0, m.pi/8, m.pi/8, m.pi/6, m.pi/6, m.pi/3, 2*m.pi/3])

#Random adjacency matrix
A = np.zeros((N,N))
for i in range(0,N-1):
    for j in range(i+1,N):
        u=r.uniform(0,1)
        if u >0.1:
            A[i][j]=2
A = A + np.transpose(A)

#Random natural frequencies
w = 2*np.random.rand(N,1)
w = w-np.mean(w)


In [109]:
#Calculate quantities from inital parameters 

edges = KFB.CalculateEdgeSet(A)
x_desired = KFB.CalculatePhaseDifferencesFromMin(x_min,edges)
D = KFB.CalculateSinMatrix(edges,x_desired)
B = KFB.CalculateIncidenceMatrix(edges,N)
B_D = np.matmul(B,D)
delta1 = KFB.CalculateDeltaFromAdj(A,edges)

In [114]:
# Construct the problem and solve the convex optimisation problem
delta = cp.Variable((len(edges),1))
beta = cp.Variable((N,1))
objective = cp.Minimize(cp.sum_squares(delta-delta1)+ cp.sum_squares(beta-w))
constraints = [B_D@delta==beta,delta >=0]
prob = cp.Problem(objective, constraints)
result = prob.solve()